In [1]:
import cv2
from mediapipe import solutions
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot

In [5]:
cap = cv2.VideoCapture(0)
detector = FaceMeshDetector(maxFaces=1)
plotY = LivePlot(640, 640, [20, 50], invert=True)

idListL = [22, 23, 24, 26, 110, 157, 158, 159, 160, 161, 130, 243]
ratioList = []
blinkCounter = 0
counter = 0
color = (255, 0, 255)

In [ ]:
idListR = [ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]

In [6]:

def check_distance(frame):
    # Convert frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces in the frame
    results = face_detection.process(frame_rgb)
    

    # Check if faces are detected
    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, ic = frame.shape
            bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                    int(bboxC.width * iw), int(bboxC.height * ih)
            cv2.rectangle(frame, bbox, (255, 0, 255), 2)

            # Calculate area of bounding box
            bbox_area = bboxC.width * bboxC.height

            # Define threshold for face distance
            distance_threshold = 0.2  # Adjust as needed

            # Check if face is too far or too close
            if bbox_area < distance_threshold:
                cv2.putText(frame, "Please come closer", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                return 0, frame
            else:
                cv2.putText(frame, "Face detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                return 1, frame

    else:
        # If no face detected, prompt user to come closer
        cv2.putText(frame, "No face detected. Please come closer", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        return 0, frame
    
while True:

    success, img = cap.read()

    mp_face_detection = solutions.face_detection
    face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.7)
    distance, img = check_distance(img)
    if distance == 0:
        cv2.imshow("Image", img)
        cv2.waitKey(25)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue
    img, faces = detector.findFaceMesh(img, draw=True)

    if faces:
        face = faces[0]
        for id in idListL:
            cv2.circle(img, face[id], 2,color, cv2.FILLED)

        leftUp = face[159]
        leftDown = face[23]
        leftLeft = face[130]
        leftRight = face[243]
        lenghtVer, _ = detector.findDistance(leftUp, leftDown)
        lenghtHor, _ = detector.findDistance(leftLeft, leftRight)

        cv2.line(img, leftUp, leftDown, (0, 200, 0), 2)
        cv2.line(img, leftLeft, leftRight, (0, 200, 0), 2)

        ratio = int((lenghtVer / lenghtHor) * 100)
        ratioList.append(ratio)
        if len(ratioList) > 3:
            ratioList.pop(0)
        ratioAvg = sum(ratioList) / len(ratioList)

        if ratioAvg < 25 and counter == 0:
            blinkCounter += 1
            color = (0,200,0)
            counter = 1
        if counter != 0:
            counter += 1
            if counter > 10:
                counter = 0
                color = (255,0, 255)

        cvzone.putTextRect(img, f'Blink Count: {blinkCounter}', (50, 100),
                        colorR=color)

        imgPlot = plotY.update(ratioAvg, color)
        img = cv2.resize(img, (640, 640))
        imgStack = cvzone.stackImages([img, imgPlot], 2, 1)
    else:
        img = cvzone.stackImages([img, img], 2, 1)


    cv2.imshow("Image", img)
    # cv2.waitKey(25)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
